In [5]:
# !pip install konlpy

     |████████████████████████████████| 19.4MB 9.1MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 460kB 58.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
train_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/df/train_df.tsv',sep='\t')
test_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/df/test_df.tsv',sep='\t')

### Tokenizer 함수 정의

In [6]:
from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    toekns_ko = okt.morphs(text)
    return toekns_ko

## TfidfVectorizer 로 학습 변환

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectoer = TfidfVectorizer(tokenizer=tw_tokenizer,ngram_range=(1,2),
                           min_df=3,max_df=0.9)

In [12]:
%time tvectoer.fit(train_df.document)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CPU times: user 11min 32s, sys: 2.15 s, total: 11min 34s
Wall time: 11min 19s


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.9, max_features=None,
                min_df=3, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tw_tokenizer at 0x7fbd6758c620>,
                use_idf=True, vocabulary=None)

In [15]:
%time X_train_tvecter = tvectoer.transform(train_df['document'])

CPU times: user 9min 26s, sys: 1.38 s, total: 9min 27s
Wall time: 9min 14s


In [18]:
%time X_test_tvecter = tvectoer.transform(test_df['document'])

CPU times: user 3min 7s, sys: 500 ms, total: 3min 8s
Wall time: 3min 4s


In [16]:
tvecter = tvectoer

In [17]:
y_train = train_df.label.values
y_test = test_df.label.values

### LogisticRegression으로 학습 평가


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression(C=3.5)
lr_clf.fit(X_train_tvecter,y_train)
pred = lr_clf.predict(X_test_tvecter)
accuracy_score(y_test,pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.859210123481988

### 실제 테스트

In [20]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ'
review2 = '이런 사랑영화가 다시 나올 수 있을까?'

In [22]:
import re
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review1)
review2 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review2)

In [26]:
review_tfvecter = tvectoer.transform([review2])
pred = lr_clf.predict(review_tfvecter)
pred[0]

1

In [29]:
reviews = ['진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ','이런 사랑영화가 다시 나올 수 있을까?']

In [30]:
review_list = []
for review in reviews:
    review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",review)
    review_list.append(review)

In [32]:
review_tfvecter = tvectoer.transform(review_list)
pred = lr_clf.predict(review_tfvecter)
pred

array([0, 1])

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
def parapipe(params, pipeline, cv=3):
    grid_pipe = GridSearchCV(pipeline, param_grid=params,cv=cv,
                            scoring='accuracy',verbose=1,n_jobs=-1)
    grid_pipe.fit(train_df.document,y_train)
    pred = grid_pipe.predict(test_df.document)
    acc = accuracy_score(y_test,pred)
    print(grid_pipe.best_params_,grid_pipe.best_score_)
    print(f'grid_pipe acc : {acc:.4f}')

In [64]:
params = {
    'tfidf_vect__ngram_range':[(1,1),(1,2)],
    'tfidf_vect__max_df':[0.9],
    'tfidf_vect__min_df':[1],
    'lr_clf__C':[3.5,4,4.5]
}
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer()),
    ('lr_clf',LogisticRegression())
])

parapipe(params, pipeline)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  2.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'lr_clf__C': 4.5, 'tfidf_vect__max_df': 0.9, 'tfidf_vect__min_df': 1, 'tfidf_vect__ngram_range': (1, 1)} 0.7950833727733536
grid_pipe acc : 0.8032656393509542


In [71]:
params = {
    'tfidf_vect__ngram_range':[(1,1)],
    'tfidf_vect__max_df':[0.6,0.7],
    'tfidf_vect__min_df':[1],
    'lr_clf__C':[4.5,5]
}
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer()),
    ('lr_clf',LogisticRegression())
])

parapipe(params, pipeline)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'lr_clf__C': 4.5, 'tfidf_vect__max_df': 0.6, 'tfidf_vect__min_df': 1, 'tfidf_vect__ngram_range': (1, 1)} 0.7950833727733536
grid_pipe acc : 0.8032656393509542


In [76]:
params = {
    'tfidf_vect__ngram_range':[(1,1),(1,2)],
    'tfidf_vect__max_df':[0.99,1],
    'tfidf_vect__min_df':[1,2,3],
    'lr_clf__C':[3.5,4,4.5]
}
pipeline = Pipeline([
    ('tfidf_vect',TfidfVectorizer()),
    ('lr_clf',LogisticRegression())
])

parapipe(params, pipeline)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  8.3min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'lr_clf__C': 4.5, 'tfidf_vect__max_df': 0.99, 'tfidf_vect__min_df': 1, 'tfidf_vect__ngram_range': (1, 1)} 0.7950833727733536
grid_pipe acc : 0.8033
